# Customer Churn Prediction - Exploratory Data Analysis

This notebook provides an interactive exploration of the Telco Customer Churn dataset and demonstrates the model building process.

## 1. Setup and Import Libraries

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_preprocessing import load_and_preprocess_data
from model_training import train_models, create_metrics_summary
from feature_analysis import identify_key_churn_drivers, analyze_churn_by_segment
from prediction import predict_churn, create_prediction_report

%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

import warnings
warnings.filterwarnings('ignore')

## 2. Load and Explore Data

In [ ]:
# Load raw data
df = pd.read_csv('../data/telco_churn.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
# Check data types and missing values
print("Data Info:")
df.info()

print("\nMissing Values:")
print(df.isnull().sum())

In [ ]:
# Churn distribution
churn_counts = df['Churn'].value_counts()
churn_pct = df['Churn'].value_counts(normalize=True) * 100

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
churn_counts.plot(kind='bar', ax=ax1, color=['skyblue', 'salmon'])
ax1.set_title('Churn Distribution (Count)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Churn')
ax1.set_ylabel('Count')
ax1.set_xticklabels(['No', 'Yes'], rotation=0)

# Pie chart
ax2.pie(churn_counts, labels=['No Churn', 'Churn'], autopct='%1.1f%%', 
        colors=['skyblue', 'salmon'], startangle=90)
ax2.set_title('Churn Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nChurn Rate: {churn_pct['Yes']:.2f}%")

## 3. Exploratory Visualizations

In [ ]:
# Churn by Contract Type
plt.figure(figsize=(10, 6))
contract_churn = pd.crosstab(df['Contract'], df['Churn'], normalize='index') * 100
contract_churn.plot(kind='bar', stacked=False, color=['skyblue', 'salmon'])
plt.title('Churn Rate by Contract Type', fontsize=14, fontweight='bold')
plt.xlabel('Contract Type')
plt.ylabel('Percentage')
plt.legend(['No Churn', 'Churn'])
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Tenure distribution by Churn
plt.figure(figsize=(12, 6))
df['tenure'].hist(by=df['Churn'], bins=30, figsize=(12, 6), 
                  color=['skyblue', 'salmon'], alpha=0.7, edgecolor='black')
plt.suptitle('Tenure Distribution by Churn Status', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Average Tenure (No Churn): {df[df['Churn']=='No']['tenure'].mean():.2f} months")
print(f"Average Tenure (Churn): {df[df['Churn']=='Yes']['tenure'].mean():.2f} months")

In [ ]:
# Monthly Charges by Churn
fig, ax = plt.subplots(figsize=(10, 6))
df.boxplot(column='MonthlyCharges', by='Churn', ax=ax, 
           patch_artist=True, 
           boxprops=dict(facecolor='skyblue', alpha=0.7))
plt.suptitle('')
plt.title('Monthly Charges Distribution by Churn', fontsize=14, fontweight='bold')
plt.xlabel('Churn')
plt.ylabel('Monthly Charges ($)')
plt.show()

## 4. Data Preprocessing

In [ ]:
# Run preprocessing pipeline
X_train, X_test, y_train, y_test, scaler = load_and_preprocess_data(
    '../data/telco_churn.csv',
    test_size=0.2,
    random_state=42
)

print(f"\nTraining set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"\nFeature count: {X_train.shape[1]}")

## 5. Model Training

In [ ]:
# Train all models
models, all_metrics = train_models(
    X_train, y_train,
    X_test, y_test,
    save_models=False
)

In [ ]:
# Display metrics summary
metrics_summary = create_metrics_summary(all_metrics)
print("\nModel Performance Summary:")
print(metrics_summary.to_string(index=False))

## 6. Feature Importance Analysis

In [ ]:
# Identify key churn drivers
feature_names = X_train.columns.tolist()
all_importance = identify_key_churn_drivers(models, feature_names, top_n=15)

In [ ]:
# Visualize XGBoost feature importance
if 'XGBoost' in all_importance and all_importance['XGBoost'] is not None:
    importance_df = all_importance['XGBoost'].head(10)
    
    plt.figure(figsize=(10, 8))
    plt.barh(range(len(importance_df)), importance_df['importance'])
    plt.yticks(range(len(importance_df)), importance_df['feature'])
    plt.xlabel('Importance Score')
    plt.title('Top 10 Features - XGBoost', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## 7. Make Predictions

In [ ]:
# Use best model for predictions
best_model = models['XGBoost']
predictions, probabilities = predict_churn(best_model, X_test)

# Create prediction report
prediction_report = create_prediction_report(X_test, predictions, probabilities)

print("\nPrediction Summary:")
print(f"Total Predictions: {len(predictions)}")
print(f"Predicted Churners: {predictions.sum()} ({predictions.mean():.2%})")
print(f"\nRisk Distribution:")
print(prediction_report['Risk_Level'].value_counts())

print("\nTop 10 High-Risk Customers:")
prediction_report.head(10)

## 8. Segment Analysis

In [ ]:
# Analyze churn by contract type
df_with_churn = df.copy()
df_with_churn['Churn'] = df_with_churn['Churn'].map({'Yes': 1, 'No': 0})

contract_analysis = analyze_churn_by_segment(df_with_churn, 'Contract', 'Churn')
print("\nChurn Analysis by Contract Type:")
print(contract_analysis)

## 9. Conclusions

Key findings from this analysis:

1. **Model Performance**: XGBoost achieved the best performance with ~91% accuracy and 0.92 AUC-ROC
2. **Top Churn Drivers**: Contract type, tenure, and monthly charges are the strongest predictors
3. **High-Risk Segments**: Month-to-month contract customers with short tenure
4. **Recommendations**: 
   - Focus retention on month-to-month customers
   - Early intervention for customers <6 months
   - Pricing review for high-charge customers

---

For production deployment, use the trained models saved in the `models/` directory.